In [4]:
class FinitFieldElement():
    def __init__(self, number:int, p:int):
        FinitFieldElement.check_errors(number, p)
        self.value = number%p
        self.p = p
        
    def __add__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value + y.value)%self.p, self.p)
    
    def __sub__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value - y.value)%self.p, self.p)
    
    def __mul__(self, y):
        self.check_field(y)
        return FinitFieldElement((self.value * y.value)%self.p, self.p)

    def __truediv__(self, y, inv_mul_method='euclidean'):
        if inv_mul_method != 'euclidean' and inv_mul_method != 'fermat':
            raise ValueError("The method to compute the multiplicative inverse must be 'euclidean' or 'fermat'")
        self.check_field(y)
            
        inv_mul_y = y.extended_euclidean_algorithm() if inv_mul_method =='euclidean' else y.fermats_little_theorem()
        return FinitFieldElement((self.value * inv_mul_y)%self.p,self.p)
    
    def __pow__(self, exponent):
        exp_aux = exponent
        while exp_aux < 0:
            exp_aux += self.p - 1
        return FinitFieldElement(pow(self.value,exp_aux,self.p), self.p)
    
    def __eq__(self, y):
        if y == None: return False
        return (self.value == y.value) and (self.p == y.p)
    
    def check_errors(number,p):
        if not isinstance(number, int):
            raise TypeError("The number must be integer")
        # if not(0 <= number < p):
        #     raise ValueError("The number must be greater than or equal to 0 and lower than the prime number P")
            
    def check_field(self, y):
        if y.p != self.p:
            raise TypeError("The numbers must belong to the same finit field")
            
    def extended_euclidean_algorithm(self):
        r0, r1 = self.value, self.p
        s0, s1 = 1, 0
        t0, t1 = 0, 1
    
        while r1 != 0:
            q = r0 // r1
            r0, r1 = r1, r0 % r1
            s0, s1 = s1, s0 - q * s1
            t0, t1 = t1, t0 - q * t1
        if r0 != 1:
            raise ValueError(f"The value {self.value} has not inverse of multiplication in {self.p}-finit field")
    
        return s0 % self.p
    
    def fermats_little_theorem(self):
        return (self.value ** (self.p - 2))%self.p

In [71]:
class EllipticCurveElement():
    def __init__(self,x,y,a,b):
        if not(x==None and y==None):
            EllipticCurveElement.check_point(x,y,a,b)
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        
    def __add__(self, other_point):
        self.check_same_curve(other_point)
        
        if self.is_inf(): return other_point
        elif other_point.is_inf(): return self
        elif (self.x==other_point.x and self.y!=other_point.y) or (self == other_point and self.y.value == 0): 
            return EllipticCurveElement(None,None,self.a,self.b)
    
        s = (other_point.y - self.y)/(other_point.x - self.x) if self!=other_point \
        else (FinitFieldElement(3, self.x.p)*(self.x**2)+self.a)/(FinitFieldElement(2, self.x.p)*self.y)
        x = s**2 - self.x - other_point.x
        y = s * (self.x - x) - self.y
        return EllipticCurveElement(x,y,self.a,self.b)
    
    def __rmul__(self, coefficient):
        coef = coefficient
        current = self
        result = self.__class__(None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result
        
    def __eq__(self, other_point):
        return (self.x == other_point.x) and (self.y == other_point.y) 
    
    
    def __ne__(self, other_point):
        return not (self==other_point)
        
    def is_inf(self):
        return (self.x==None) and (self.y==None)
    
    def get_group_order(self):
        result=self
        k=1
        while result.x != None or result.y != None:
            result += self
            k+=1
        return k
            
    def check_same_curve(self, other_point):
        if self.a != other_point.a or self.b != other_point.b:
            raise ValueError(f"(The given points are not in the same elliptic curve")

    def check_point(x,y,a,b):
        if (x==None) ^ (y==None):
            raise ValueError(f"Invalid point")
        # if abs((y**2) - (x**3 + a * x + b)) > 0.000001:
        #     raise ValueError(f"The point ({x},{y}) is not on the elliptic curve")            
        if y**2 != x**3 + a * x + b:
            raise ValueError(f"The point ({x},{y}) is not on the elliptic curve")
        

In [ ]:
p1 = EllipticCurveElement(0,2.64575131106459059,0,7)
p2 = EllipticCurveElement(0,-2.64575131106459059,0,7)

print((p1+p2).y)

In [6]:
prime = 223
a = FinitFieldElement(0, prime)
b = FinitFieldElement(7, prime)
x = FinitFieldElement(47, prime)
y = FinitFieldElement(71, prime)
p = EllipticCurveElement(x, y, a, b)
print("llega")
for s in range(1,21):
    result = s*p
    print('{}*(47,71)=({},{})'.format(s,result.x.value,result.y.value))


llega
1*(47,71)=(47,71)
2*(47,71)=(36,111)
3*(47,71)=(15,137)
4*(47,71)=(194,51)
5*(47,71)=(126,96)
6*(47,71)=(139,137)
7*(47,71)=(92,47)
8*(47,71)=(116,55)
9*(47,71)=(69,86)
10*(47,71)=(154,150)
11*(47,71)=(154,73)
12*(47,71)=(69,137)
13*(47,71)=(116,168)
14*(47,71)=(92,176)
15*(47,71)=(139,86)
16*(47,71)=(126,127)
17*(47,71)=(194,172)
18*(47,71)=(15,86)
19*(47,71)=(36,112)
20*(47,71)=(47,152)


In [7]:
def is_in_curve(x,y,a,b):
    return y**2 == x**3 + a * x + b

p = 1021
a = FinitFieldElement(-3,p)
b = FinitFieldElement(-3,p)
n_points = 0
for i in range(p):
    for j in range(p):
        x = FinitFieldElement(i,p)
        y = FinitFieldElement(j,p)
        if is_in_curve(x,y,a,b):
            n_points +=1

n_points+=1 #Infinity
print(n_points)

1039


In [16]:
def is_in_curve(x,y,a,b):
    return y**2 == x**3 + a * x + b

p = 17
a = FinitFieldElement(2,p)
b = FinitFieldElement(2,p)
n_points = 0
for i in range(p):
    for j in range(p):
        x = FinitFieldElement(i,p)
        y = FinitFieldElement(j,p)
        if is_in_curve(x,y,a,b):
            n_points +=1

n_points+=1 #Infinity
print(n_points)

19


In [8]:
# Usando P=(379,1011), obtener kP, siendo k=655.

x=FinitFieldElement(379,p)
y=FinitFieldElement(1011,p)
point=EllipticCurveElement(x,y,a,b)
k=655

result = k*point
print(f"({result.x.value},{result.y.value})")

(388,60)


In [72]:
x=FinitFieldElement(0,73)
y=FinitFieldElement(72,73)
a=FinitFieldElement(1,73)
b=FinitFieldElement(1,73)
point=EllipticCurveElement(x,y,a,b)

result=point
res_prev=point
k=1
while result.x != None or result.y != None:
    res_prev=result
    result += point
    k+=1
    
print(f"Listo: ({result.x},{result.y}) - k {k} - prev: ({res_prev.x.value},{res_prev.y.value})")
point.get_group_order()

Listo: (None,None) - k 36 - prev: (0,1)


36

- Seleccionar clave privada
    - Calcular orden del grupo del generador
    - Generar numero random entre 0 y orden-1
- Generar clave publica: a*G
- Enviar clave publica:
- Recibir clave publica ajena: b*G
- Generar clave compartida: a*b*G

In [188]:
p=43
a=FinitFieldElement(0,p)
b=FinitFieldElement(6,p)
x=FinitFieldElement(9,p)
y=FinitFieldElement(2,p)
g = EllipticCurveElement(x,y,a,b)
g.get_group_order()

39

In [189]:
import socket
import random
from multiprocessing import Process, Lock
import time
import pickle

port = random.randint(1024,65535)
bob_addr = ('127.0.0.1',port)

#Parametros conocidos
p=43
a=FinitFieldElement(0,p)
b=FinitFieldElement(6,p)
x=FinitFieldElement(13,p)
y=FinitFieldElement(15,p)
g = EllipticCurveElement(x,y,a,b)
print("----- ECDH -----")
print(f"p:{p}\nG:{(g.x.value,g.y.value)}\nEC: y²=x³+{a.value}x+{b.value}")
print("----------------\n")
def diffie_hellman(g, socket,name):
    log_msg = []
    n_order = g.get_group_order()
    
    sk = FinitFieldElement(random.randint(1,n_order-1),p)
    pk = (sk.value)*g
    msg = pickle.dumps(pk)
    socket.send(msg)
    
    msg_rcv = socket.recv(1024)
    pk_rcv = pickle.loads(msg_rcv)
    shared_key = (sk.value)*pk_rcv
    
    log_msg.append(f"[{name}] My SK: {sk.value}")
    log_msg.append(f"[{name}] My PK: ({pk.x.value},{pk.y.value})")
    log_msg.append(f"[{name}] PK received: ({pk_rcv.x.value},{pk_rcv.y.value})")
    log_msg.append(f"[{name}] The obtained key is: {shared_key.x.value}\n")
    return shared_key, log_msg

def alice(g, bob_addr,lock):
    skt = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    skt.connect(bob_addr)
    
    shared_key, log = diffie_hellman(g,skt,'ALICE')

    lock.acquire()
    print(*log,sep='\n')
    lock.release()
    skt.close()
    
def bob(g, bob_addr,lock):    
    wlc_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    wlc_socket.bind(bob_addr)
    wlc_socket.listen(0)
    skt, _ = wlc_socket.accept()
    
    shared_key, log = diffie_hellman(g,skt,'BOB')
    
    lock.acquire()
    print(*log,sep='\n')
    lock.release()
    skt.close()
    
    
lock = Lock()
bob_thread = Process(target=bob,args=(g,bob_addr,lock))
bob_thread.start()
time.sleep(0.1)
alice_thread = Process(target=alice, args=(g,bob_addr,lock))
alice_thread.start()

bob_thread.join()
alice_thread.join()

----- ECDH -----
p:43
G:(13, 15)
EC: y²=x³+0x+6
----------------

[BOB] My SK: 6
[BOB] My PK: (27,34)
[BOB] PK received: (13,28)
[BOB] The obtained key is: 27

[ALICE] My SK: 12
[ALICE] My PK: (13,28)
[ALICE] PK received: (27,34)
[ALICE] The obtained key is: 27



In [82]:
n_order = g.get_group_order()
for i in range(n_order+1):
    res = i*g
    if res.x == None and res.y == None:
        print(f"{i} - infinito")
    else:
        print(f"{i} - ({res.x.value},{res.y.value})")

0 - infinito
1 - (13,15)
2 - (33,34)
3 - (38,15)
4 - (35,28)
5 - (26,34)
6 - (27,34)
7 - (27,9)
8 - (26,9)
9 - (35,15)
10 - (38,28)
11 - (33,9)
12 - (13,28)
13 - infinito


In [187]:
i=1
k=1

for i in range(1,15):
    i_f = FinitFieldElement(i,n_order)
    for j in range(k,15):
        j_f = FinitFieldElement(j, n_order)
        res = (i_f.value)*(j_f.value)*g
        #res = (i)*(j)*g
        if res.x==None:
            print(f"{i}*{j}*g = infinito")
        else:
            print(f"{i}*{j}*g = {res.x.value,res.y.value}")
    print({i,j})
    k+=1

1*1*g = (13, 15)
1*2*g = (33, 34)
1*3*g = (38, 15)
1*4*g = (35, 28)
1*5*g = (26, 34)
1*6*g = (27, 34)
1*7*g = (27, 9)
1*8*g = (26, 9)
1*9*g = (35, 15)
1*10*g = (38, 28)
1*11*g = (33, 9)
1*12*g = (13, 28)
1*13*g = infinito
1*14*g = (13, 15)
{1, 14}
2*2*g = (35, 28)
2*3*g = (27, 34)
2*4*g = (26, 9)
2*5*g = (38, 28)
2*6*g = (13, 28)
2*7*g = (13, 15)
2*8*g = (38, 15)
2*9*g = (26, 34)
2*10*g = (27, 9)
2*11*g = (35, 15)
2*12*g = (33, 9)
2*13*g = infinito
2*14*g = (33, 34)
{2, 14}
3*3*g = (35, 15)
3*4*g = (13, 28)
3*5*g = (33, 34)
3*6*g = (26, 34)
3*7*g = (26, 9)
3*8*g = (33, 9)
3*9*g = (13, 15)
3*10*g = (35, 28)
3*11*g = (27, 9)
3*12*g = (38, 28)
3*13*g = infinito
3*14*g = (38, 15)
{3, 14}
4*4*g = (38, 15)
4*5*g = (27, 9)
4*6*g = (33, 9)
4*7*g = (33, 34)
4*8*g = (27, 34)
4*9*g = (38, 28)
4*10*g = (13, 15)
4*11*g = (26, 34)
4*12*g = (35, 15)
4*13*g = infinito
4*14*g = (35, 28)
{4, 14}
5*5*g = (13, 28)
5*6*g = (35, 28)
5*7*g = (35, 15)
5*8*g = (13, 15)
5*9*g = (27, 34)
5*10*g = (33, 9)
5*11*g 